In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option("display.max_columns", None)
data = pd.read_csv("Data/preprocessed_data.csv", delimiter=",")
data.head(20)

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,Admission grade bins,Age group,Unemployment rate bins
0,1,17,5,171,1,1,122.0,1.0,19,12,5,9,127.3,1,0,0,1,1.0,0,20.0,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout,NaN,18-22,10-15%
1,1,15,1,9254,1,1,160.0,1.0,1,3,3,3,142.5,1,0,0,0,1.0,0,19.0,0,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate,NaN,18-22,10-15%
2,1,1,5,9070,1,1,122.0,1.0,37,37,9,9,124.8,1,0,0,0,1.0,0,19.0,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout,NaN,18-22,10-15%
3,1,17,2,9773,1,1,122.0,1.0,38,37,5,3,119.6,1,0,0,1,0.0,0,20.0,0,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate,NaN,18-22,5-10%
4,2,39,1,8014,0,1,100.0,1.0,37,38,9,9,141.5,0,0,0,1,0.0,0,34.0,0,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate,NaN,28-35,10-15%
5,2,39,1,9991,0,19,133.1,1.0,37,37,9,7,114.8,0,0,1,1,1.0,0,34.0,0,0,5,10,5,11.857143,0,0,5,17,5,11.500000,5,16.2,0.3,-0.92,Graduate,NaN,28-35,15%+
6,1,1,1,9500,1,1,142.0,1.0,19,38,7,10,128.4,1,0,0,1,0.0,1,18.0,0,0,7,9,7,13.300000,0,0,8,8,8,14.345000,0,15.5,2.8,-4.06,Graduate,NaN,18-22,15%+
7,1,18,4,9254,1,1,119.0,1.0,37,37,9,9,113.1,1,0,0,0,1.0,0,22.0,0,0,5,5,0,0.000000,0,0,5,5,0,0.000000,0,15.5,2.8,-4.06,Dropout,NaN,18-22,15%+
8,1,1,3,9238,1,1,137.0,62.0,1,1,9,9,129.3,0,0,0,1,0.0,1,21.0,1,0,6,8,6,13.875000,0,0,6,7,6,14.142857,0,16.2,0.3,-0.92,Graduate,NaN,18-22,15%+
9,1,1,1,9238,1,1,138.0,1.0,1,19,4,7,123.0,1,0,1,0,0.0,0,18.0,0,0,6,9,5,11.400000,0,0,6,14,2,13.500000,0,8.9,1.4,3.51,Dropout,NaN,18-22,5-10%


In [8]:
# Preprocessing

# Encoding only categorical columns
categorical_columns = [
    'Marital status', 'Application mode', 'Application order', 'Course',
    'Daytime/evening attendance', 'Previous qualification', 'Nacionality',
    "Mother's qualification", "Father's qualification", "Mother's occupation",
    "Father's occupation", 'Displaced', 'Educational special needs', 'Debtor',
    'Tuition fees up to date', 'Gender', 'Scholarship holder', 'International', 'Target',
    'Age group', 'Unemployment rate bins'
]

for col in categorical_columns:
    data[col] = data[col].astype(str)

# One hot encoding
encoded_data = pd.get_dummies(data)

# Dropping all non categorical columns
encoded_data = encoded_data.drop(columns=["Previous qualification (grade)", "Admission grade",
                           "Age at enrollment", "Curricular units 1st sem (credited)",
                           "Curricular units 1st sem (enrolled)", "Curricular units 1st sem (evaluations)",
                           "Curricular units 1st sem (approved)", "Curricular units 1st sem (grade)",
                           "Curricular units 1st sem (without evaluations)", "Curricular units 2nd sem (credited)",
                           "Curricular units 2nd sem (enrolled)", "Curricular units 2nd sem (evaluations)",
                           "Curricular units 2nd sem (approved)", "Curricular units 2nd sem (grade)",
                           "Curricular units 2nd sem (without evaluations)", "Unemployment rate",
                           "Inflation rate", "GDP"])

encoded_data.head(20)

,Marital status_1,Marital status_2,Marital status_3,Marital status_4,Marital status_5,Marital status_6,Application mode_1,Application mode_10,Application mode_15,Application mode_16,Application mode_17,Application mode_18,Application mode_2,Application mode_26,Application mode_27,Application mode_39,Application mode_42,Application mode_43,Application mode_44,Application mode_5,Application mode_51,Application mode_53,Application mode_57,Application mode_7,Application order_0,Application order_1,Application order_2,Application order_3,Application order_4,Application order_5,Application order_6,Application order_9,Course_171,Course_33,Course_8014,Course_9003,Course_9070,Course_9085,Course_9119,Course_9130,Course_9147,Course_9238,Course_9254,Course_9500,Course_9556,Course_9670,Course_9773,Course_9853,Course_9991,Daytime/evening attendance_0,Daytime/evening attendance_1,Previous qualification_1,Previous qualification_10,Previous qualification_12,Previous qualification_14,Previous qualification_15,Previous qualification_19,Previous qualification_2,Previous qualification_3,Previous qualification_38,Previous qualification_39,Previous qualification_4,Previous qualification_40,Previous qualification_42,Previous qualification_43,Previous qualification_5,Previous qualification_6,Previous qualification_9,Nacionality_1.0,Nacionality_100.0,Nacionality_101.0,Nacionality_103.0,Nacionality_105.0,Nacionality_108.0,Nacionality_109.0,Nacionality_11.0,Nacionality_13.0,Nacionality_14.0,Nacionality_17.0,Nacionality_2.0,Nacionality_21.0,Nacionality_22.0,Nacionality_24.0,Nacionality_25.0,Nacionality_26.0,Nacionality_32.0,Nacionality_41.0,Nacionality_6.0,Nacionality_62.0,Nacionality_nan,Mother's qualification_1,Mother's qualification_10,Mother's qualification_11,Mother's qualification_12,Mother's qualification_14,Mother's qualification_18,Mother's qualification_19,Mother's qualification_2,Mother's qualification_22,Mother's qualification_26,Mother's qualification_27,Mother's qualification_29,Mother's qualification_3,Mother's qualification_30,Mother's qualification_34,Mother's qualification_35,Mother's qualification_36,Mother's qualification_37,Mother's qualification_38,Mother's qualification_39,Mother's qualification_4,Mother's qualification_40,Mother's qualification_41,Mother's qualification_42,Mother's qualification_43,Mother's qualification_44,Mother's qualification_5,Mother's qualification_6,Mother's qualification_9,Father's qualification_1,Father's qualification_10,Father's qualification_11,Father's qualification_12,Father's qualification_13,Father's qualification_14,Father's qualification_18,Father's qualification_19,Father's qualification_2,Father's qualification_20,Father's qualification_22,Father's qualification_25,Father's qualification_26,Father's qualification_27,Father's qualification_29,Father's qualification_3,Father's qualification_30,Father's qualification_31,Father's qualification_33,Father's qualification_34,Father's qualification_35,Father's qualification_36,Father's qualification_37,Father's qualification_38,Father's qualification_39,Father's qualification_4,Father's qualification_40,Father's qualification_41,Father's qualification_42,Father's qualification_43,Father's qualification_44,Father's qualification_5,Father's qualification_6,Father's qualification_9,Mother's occupation_0,Mother's occupation_1,Mother's occupation_10,Mother's occupation_122,Mother's occupation_123,Mother's occupation_125,Mother's occupation_131,Mother's occupation_132,Mother's occupation_134,Mother's occupation_141,Mother's occupation_143,Mother's occupation_144,Mother's occupation_151,Mother's occupation_152,Mother's occupation_153,Mother's occupation_171,Mother's occupation_173,Mother's occupation_175,Mother's occupation_191,Mother's occupation_192,Mother's occupation_193,Mother's occupation_194,Mother's occupation_2,Mother's occupation_3,Mother's occupation_4,Mother's occupation_5,Mother's occupation_6,Mother's occupation_7,Mother's occupation_8,Mother's o

In [59]:
# Mining Frequent Itemsets

def apriori_itemsets(data, support, maxlen=10):
    # use sets
    candidates = []
    frequent = []
    all_frequent = []
    min_support_count = support*data.shape[0]
    k = 1
    
    # Step 0
    # loop through columns and generate candidate k = 1 itemsets
    for column in data:
        itemset = {column}
        itemset = frozenset(itemset)
        candidates.append(itemset)
    # Check if each is frequent, if it is add it to frequent
    for candidate in candidates:
        support_count = 0
        # can't index sets so get the item out of the candidate set like this
        for item in candidate:
            break
        for transaction in data.loc[:,item]:
            if transaction:
                support_count += 1
        if support_count >= min_support_count:
            frequent.append(candidate)
    all_frequent = all_frequent + frequent

    # Main loop
    # repeat until frequent is empty or k=max
    while frequent != [] and k < maxlen:
        candidates = []
        
        # Loop through frequent and generate candidate k+1 itemsets by combining k itemsets
        for i in range(len(frequent) - 1):
            for j in range(i+1, len(frequent)):
                new_itemset = frequent[i].union(frequent[j])
                candidates.append(new_itemset)
        candidates = set(candidates)
        
        # Remove old frequents
        frequent = []
        
        # Check if each is frequent and add it to frequent
        for candidate in candidates:
            support_count = 0
            for index, row in data.iterrows():
                present = True
                for item in candidate:
                    if not row[item]:
                        present = False
                        break
                if present:
                    support_count += 1
            if support_count >= min_support_count:
                frequent.append(candidate)
        all_frequent = all_frequent + frequent
        k += 1
    return all_frequent
        

In [ ]:
# Call function
frequent_itemsets = apriori_itemsets(encoded_data, support=0.05, maxlen=3)
print(frequent_itemsets)

In [ ]:
# Mining Frequent Rules

class AprioriRule:
    # Class for an association rule
    def __init__(self):
        self.antecedent
        self.consequent
        self.confidence

def AprioriRules(frequent_itemsets, confidence):
    rules = []
    all_rules = []

    # loop through itemsets and generate all non-empty subsets
    # generate all rules using, "s -> (I - s)" checking if, support of I / support of s > min confidence
    
    return all_rules